In [19]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, adjusted_rand_score

Мы будем использовать данные, взятые с датчиков акселерометров и гироскопов смартфонов Samsung Galaxy S3. Телефоны носили в кармане добровольцы в возрасте от 19 до 49 лет. Смартфоны постоянно фиксировали значения ускорения и скорости по трём измерениям, а поведение людей записывали на видео, чтобы вручную отметить, какую физическую активность осуществлял человек в тот или иной момент.

Данные содержат следующие признаки:

* различные показатели с акселерометра и гироскопа;
* метка активности (физическая активность человека в конкретный момент).

Попробуем на основе данных с гироскопа и акселерометра разделить активности людей на некоторые схожие по своим характеристикам группы. В идеале наблюдения во время ходьбы должны попасть в один кластер, наблюдения во время подъёма по лестнице — в другой и т. д.

In [8]:
X_train = np.loadtxt("data/train.txt")
y_train = np.loadtxt("data/train_labels.txt")
 
X_test = np.loadtxt("data/test.txt")
y_test = np.loadtxt("data/test_labels.txt")
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))
X.shape

(10299, 561)

In [9]:
len(np.unique(y))

6

Как видите, есть ряд активностей, обозначенных цифрами. Эти метки означают следующее:

- 1 — ходьба;
- 2 — подъём;
- 3 — спуск;
- 4 — сидение;
- 5 — стояние;
- 6 — лежание.

In [10]:
#определяем скейлер
scaler = StandardScaler()
#масштабируем данные
X_scaled = scaler.fit_transform(X)
#выводим нужное значение
X_scaled[0][0]

0.2105338541767611

### Задание 4.4

Пора переходить к кластеризации. Для начала определите оптимальное количество кластеров, используя внутренние меры кластеризации. Используйте все известные вам коэффициенты, реализуемые в библиотеке sklearn: коэффициент силуэта, индекс Калински — Харабаса и индекс Дэвиса — Болдина. В качестве алгоритма возьмите k-means++, в качестве значения параметра random_state — число 42.

Выведите оптимальное количество кластеров для каждой метрики, перебирая значения от 2 до 9 включительно. Также введите значение каждой метрики, округлённое до двух знаков после точки-разделителя.

In [11]:
metric_list_k_means = []

#проходимся циклом по всем нужным нам значениям количества кластеров
for i in range(2,10):
	# обучаем модель k-means с текущим количеством кластеров
    km = KMeans(n_clusters = i , random_state = 42 ).fit(X_scaled)
	#предсказываем метки
    y_pred = km.labels_
    
    sil_score = metrics.silhouette_score(X_scaled,  y_pred)
    calinski_score = metrics.calinski_harabasz_score(X_scaled,y_pred)
    davies_score = metrics.davies_bouldin_score(X_scaled, y_pred)
    
    metric_list_k_means.append((i, sil_score, davies_score, calinski_score))
	# #выводим результат для коэффициента силуэта
    # print('Результат для коэффициента силуэта')
    # print (i, metrics.silhouette_score(X_scaled,  y_pred))
    # print('___________________')    
    # print('Результат для коэффициента Калински-Харабаса')
    # print (i, metrics.calinski_harabasz_score(X_scaled,y_pred))
    # print('___________________')
    # print('Результат для коэффициента Дэвиса-Болдина')
    # print (i, metrics.davies_bouldin_score(X_scaled, y_pred))    
    
df_metric_k_means = pd.DataFrame(metric_list_k_means, columns=['number_of_clusters', 'siluet', 'calinski_harabasz', 'davies_bouldin'])    
df_metric_k_means

,number_of_clusters,siluet,calinski_harabasz,davies_bouldin
0,2,0.393732,1.070744,7880.813904
1,3,0.315484,1.786516,5034.475257
2,4,0.150529,2.340930,3696.338198
3,5,0.127237,2.431375,3027.076172
4,6,0.110969,2.367036,2556.773574
5,7,0.085419,2.681980,2216.563937
6,8,0.076183,2.611226,1974.971496
7,9,0.076488,2.581888,1790.953173


In [13]:
#определяем модель k-means
model= KMeans(n_clusters=6, init='random', random_state=42)
#обучаем модель
model.fit(X_scaled)
#вычисляем значение однородности
print('Однородность')
print(homogeneity_score(y, model.labels_))
print('')
#вычисляем значение полноты
print('Полнота')
print(completeness_score(y, model.labels_))
print('')
#вычисляем значение скорректированного индекса Рэнда
print('Индекс Ренда')
print(adjusted_rand_score(y, model.labels_))

Однородность
0.5404114294595578

Полнота
0.5809491951515007

Индекс Ренда
0.4196031125923396


In [14]:
#создаём таблицу сопряжённости
ct = pd.crosstab(y, model.labels_)
#определяем название активностей
ct.index = ['ходьба', 'подъём', 
             'спуск', 'сидение', 'стояние', 'лежание']
ct.columns = list(range(1,7))
ct

,1,2,3,4,5,6
ходьба,0,0,903,78,741,0
подъём,0,0,1242,5,295,2
спуск,0,0,321,196,889,0
сидение,91,1238,1,0,0,447
стояние,0,1346,0,0,0,560
лежание,1556,54,5,0,0,329


In [15]:
#определяем модель k-means
km = KMeans(n_clusters=2, init='random', random_state=42)
#обучаем модель
km.fit(X_scaled)

KMeans(init='random', n_clusters=2, random_state=42)

In [16]:
ctable = pd.crosstab(y, km.labels_)
ctable.index = ['ходьба', 'подъём', 
             'спуск', 'сидение', 'стояние', 'лежание']
ctable.columns = list(range(1,3))
ctable

,1,2
ходьба,1722,0
подъём,1536,8
спуск,1406,0
сидение,3,1774
стояние,0,1906
лежание,12,1932


In [17]:
metrics.completeness_score(y, km.labels_)

0.979530559699631

In [20]:
ag = AgglomerativeClustering(n_clusters=2).fit(X_scaled)
print("Completeness: %0.3f" % metrics.completeness_score(y, ag.labels_))

Completeness: 1.000
